##  music21 exercise 1
Q. **Are there melodic changes between the three renditions in the performance of an Arab-Andalusian *mīzān*, namely *muassa‘*, *mahzūz* and *inṣirāf*?**

M. *Plot a bar chart with the number of intervals in each of these three sections, taken from the four scores segmented in the annotations csv file.*

In [ ]:
from music21 import *
import os
import numpy as np
import matplotlib.pyplot as plt

First, create a dictionary from the annotations csv file with starting and ending offset of the segments that should be analysed. The keys of this dictionary are the first two letters of each section, namely `mu` for _muassa‘_, `ma` for _mahzūz_ and `in` for _inṣirāf_. The value of these keys will be another dictionary, where the keys are the mbid of the score that contatins any of these sections, and the values are lists containing the starting and ending offset of the corresponding section in that score.

In order to extract this information, is important to carefully study and understand the annotations csv file. The interesting information is located in the first column for the mbid (the name of each score is precisely this mbdi), the second column for the section label, and columns 5 and 6 for the starting and ending offset of each section. The csv file offers annotations per each section of lyrics lines, but for this exercise the relevant information is the tempo section, which is indicated with the first to letters of the labels in the second column. Therefore, to see when a tempo section changes, look for a change of these two initial letters, and take the starting offset of this row as the start of the tempo section and the ending offset of the previous row as the end of the previous tempo section.

In [ ]:
csvFile = 'arab_andalusian_lines.csv'

with open(csvFile, 'r', encoding='utf-8') as f:
    data = f.readlines()

sections = {'mu': {}, 'ma': {}, 'in': {}}

currentSection = '' # To keep track of changes of tempo sections

for i in range(1, len(data)):
    # Information from the current row
    mbid = data[i].split(',')[0]
    section = data[i].split(',')[1][:2]
    start = data[i].split(',')[5]
    # Information from the previous row
    previousMbid = data[i-1].split(',')[0]
    previousSection = data[i-1].split(',')[1][:2]
    previousEnd = data[i-1].split(',')[6]
    # Check if there is a change in the tempo section
    if currentSection != section:
        currentSection = section
        sections[currentSection][mbid] = [start]
        if i > 1:
            sections[previousSection][previousMbid].append(previousEnd)

# Add the ending offset of the last tempo section from the last row
mbid = data[-1].split(',')[0]
section = data[-1].split(',')[1][:2]
end = data[-1].split(',')[6]
sections['in'][mbid].append(end)

# Print the dictionary to verify the completeness of the results
for section in sections:
    print(section)
    for mbid in sections[section]:
        print(' ', mbid, ':', sections[section][mbid])

To compute the intervals, retrieve all the score sections contained in the dictionary per each mbid in each tempo section. Count them per tempo section.

In [ ]:
# General dictionary that will include a key per tempo section
intervals = {}

for section in sections:
    print('\nWorking with section', section)
    intervals[section] = {} # Dictionary of the tempo section
    for mbid in sections[section]: # 
        s = converter.parse('ArabAndalusianScores-selection/' + mbid + '.xml')
        print(mbid + '.xml loaded')
        p = s.parts[0]
        nn = p.flat.notesAndRests.stream()
        start = float(sections[section][mbid][0])
        end = float(sections[section][mbid][1])
        sectionNotes = nn.getElementsByOffset(start, end).stream() # All the notes in the current section
        for n in sectionNotes[:-1]:
            if n.isNote and n.next().isNote:
                itv = interval.Interval(n, n.next())
                intervals[section][itv.name] = intervals[section].get(itv.name, 0) + 1

Plot a figure with an interval histogram per tempo section. Before plotting each histogram, order the intervals according to their semitones size. For better comparison, normalize the y axis values for all histograms and give common x and y axes limits.

In [ ]:
plt.figure()

sectionsOrder = ['mu', 'ma', 'in']
for s in sectionsOrder:
    sectionDic = intervals[s]
    # Order the intervals
    # Create a dictionary with the equivalence of each interval's size in semitones and its name.
    intervalsOrder = {}
    for k in sectionDic.keys():
        itv = interval.Interval(k)
        intervalsOrder[itv.semitones] = k
    # Ordered list of intervals by semitones size
    xValues = sorted(intervalsOrder.keys())
    # Oredred list of interval names by their semitiones size to be use as ticks for the x axis.
    xTicks = [intervalsOrder[i] for i in xValues]
    # Ordered list of y axis values
    yValues = np.array([sectionDic[i] for i in xTicks])
    # Normalize yValues for better comparison
    yValues = yValues / sum(yValues)
    
    # Create the subplot
    plt.subplot(311 + sectionsOrder.index(s) )
    plt.bar(xValues, yValues)
    plt.xticks(xValues, xTicks)
    # Common x and y axes limits
    plt.xlim(-1, 13)
    plt.ylim(0, 0.5)
    plt.title(s)

plt.tight_layout()
plt.show()